# Explore here

It's recommended to use this notebook for exploration purposes.

In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd
import requests
import re

url = "https://www.finance.yahoo.com/most-active"
response = requests.get(url)

def scrape_new_url(url):
    if response.status_code != 200:
        raise Exception(f"Error al acceder a la URL: {url}")
    print(response)

scrape_new_url(url)


<Response [200]>


In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")    
soup    

<!DOCTYPE html>
<html class="NoJs chrome desktop failsafe" data-color-theme="light" id="atomic" lang="en-US"><head prefix="og: https://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charset="utf-8"/><title>Most Active Stocks Today - Yahoo Finance</title><meta content="Stock Screener, industry, index membership, share data, stock price, market cap, beta, sales, profitability, valuation ratios, analyst estimates, large cap value, bargain growth, preset stock screens" name="keywords"/><meta content="on" http-equiv="x-dns-prefetch-control"/><meta content="on" property="twitter:dnt"/><meta content="458584288257241" property="fb:app_id"/><meta content="#400090" name="theme-color"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="See the list of the most active stocks today, including share price change and percentage, trading volume, intraday highs and lows, and day charts." lang="en-

In [3]:
tables = soup.find_all('table')
tables

#Encuentra en el codigo de la pagina cualquier grupo de informacion que tenga la caracteristica 

[<table class="W(100%)"><thead><tr class="C($tertiaryColor) BdB Bdbc($seperatorColor)"><th class="Ta(start) Pstart(6px) Pend(10px) Miw(90px) Fz(xs) Py(5px)! Bgc($lv3BgColor) Va(m) Cur(p) Bgc($hoverBgColor):h Fw(400)! Ta(start) Start(0) Pend(10px) Pos(st) Bgc($lv3BgColor) Z(1) Ta(start)!"><label class="Ta(c) Pos(r) Va(tb) Pend(5px) D(n)--print" data-id=""><input aria-label="Select All" class="Pos(a) Op(0) checkbox" type="checkbox"/><svg class="Va(m)! H(16px) W(16px) checkbox:f+Stk($linkColor)! checkbox:f+Fill($linkColor)! Stk($plusGray) Fill($plusGray) Cur(p)" data-icon="checkbox-unchecked" height="16" style="stroke-width:0;vertical-align:bottom" viewbox="0 0 24 24" width="16"><path d="M3 3h18v18H3V3zm19-2H2c-.553 0-1 .448-1 1v20c0 .552.447 1 1 1h20c.552 0 1-.448 1-1V2c0-.552-.448-1-1-1z"></path></svg></label>Symbol<div class="W(3px) Pend(5px) Pos(a) Start(100%) T(0) H(100%) Bg($pfColumnFakeShadowGradient) Pe(n)"></div></th><th class="Ta(start) Px(10px) Fz(xs) Py(5px)! Bgc($lv3BgColor) 

In [4]:
#Encuentra una tipo de tabla especifica dentro de la sopa especificada anteriormente para crear el indice

for index, table in enumerate(tables):
    if ("span" in str(table)):
        table_index = index
        break

active_stocks = pd.DataFrame(columns = ["Symbol", "Name", "Price", "Change", "PercentualChange"]) #inicia el dataframe con los nombres de las columnas
for row in tables[table_index].tbody.find_all("tr"): #busca las filas debajo de cada columna especificada en la tabla, contiene la clase y la informacion en sentido vertical que le sigue
    col = row.find_all("td") #para llamar correctamente la informacion se llama con su etiqueta td
    if (col != []): #Asignando posiciones en el dataframe
        Symbol = col[0].text
        Name = col[1].text
        Price = col[2].text
        Change = col[3].text.replace("+", "")
        PercentualChange = col[4].text.replace("+", "").replace("%", "")#en algunos es preciso eliminar simbolos
        active_stocks = pd.concat([active_stocks, pd.DataFrame({ #sumo el dataframe original con uno nuevo que contiene respectivamente los nombres asignados que concuerdan y el contenido de sus filas
            "Symbol": Symbol,
            "Name": Name,
            "Price": Price,
            "Change": Change,
            "PercentualChange": PercentualChange
        }, index = [0])], ignore_index = True)

active_stocks.head(16)

,Symbol,Name,Price,Change,PercentualChange
0,SOUN,"SoundHound AI, Inc.",7.84,-0.49,-5.91
1,TSLA,"Tesla, Inc.",175.18,3.86,2.25
2,AMD,"Advanced Micro Devices, Inc.",177.83,-3.59,-1.98
3,MARA,"Marathon Digital Holdings, Inc.",21.69,2.37,12.27
4,INTC,Intel Corporation,41.83,-0.22,-0.52
5,NIO,NIO Inc.,5.11,-0.09,-1.64
6,PDD,PDD Holdings Inc.,132.00,4.32,3.38
7,F,Ford Motor Company,12.82,0.52,4.27
8,SOFI,"SoFi Technologies, Inc.",7.24,0.32,4.68
9,NVDA,NVIDIA Corporation,899.32,5.34,0.60
